In [2]:
import cv2
import numpy as np
import os
from datetime import datetime

In [46]:
video_path = 'D:\System 69\Downloads\dre.mp4'  
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video.")
    exit()


In [43]:
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))


In [42]:
quadrant_width = frame_width // 2
quadrant_height = frame_height // 2

quadrants = {
    3: [(0, 0), (quadrant_width, quadrant_height)],
    4: [(quadrant_width, 0), (frame_width, quadrant_height)],
    2: [(0, quadrant_height), (quadrant_width, frame_height)],
    1: [(quadrant_width, quadrant_height), (frame_width, frame_height)]
}


In [6]:
color_ranges = {
    'yellow-ish': ([25, 100, 50], [35, 255, 255]),   
    'blue-ish': ([80, 50, 50], [100, 255, 255]),     
    'orange-ish': ([5, 150, 100], [15, 255, 255]) ,  
    'white': ([0, 0, 200], [180, 30, 255])           
}

In [7]:
def get_quadrant(x, y, quadrants):
    for quad_num, ((x1, y1), (x2, y2)) in quadrants.items():
        if x1 <= x < x2 and y1 <= y < y2:
            return quad_num
    return None




In [8]:
def draw_quadrants(frame, quadrants):
   
    for quad_num, ((x1, y1), (x2, y2)) in quadrants.items():
        # Draw rectangle
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
        # Display quadrant number
        text_position = ((x1 + x2) // 2, (y1 + y2) // 2)
        cv2.putText(frame, f"Q{quad_num}", text_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

In [9]:
def detect_ball_color(hsv_frame, color_ranges):
    for color, (lower, upper) in color_ranges.items():
        lower_bound = np.array(lower, dtype="uint8")
        upper_bound = np.array(upper, dtype="uint8")
        mask = cv2.inRange(hsv_frame, lower_bound, upper_bound)
        if cv2.countNonZero(mask) > 0:
            return color
    return None


In [38]:

def display_event_text(frame, quad_num, event):
    corner_x = quadrants[quad_num][0][0] + 10
    corner_y = quadrants[quad_num][0][1] + 30
    cv2.putText(frame, event, (corner_x, corner_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

In [10]:
def refine_mask(mask):
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    return mask

In [47]:

event_log = []
ball_positions = {}


while True:
    ret, frame = cap.read()
    if not ret:
        break

    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    timestamp = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0  
    draw_quadrants(frame, quadrants)

    for color, (lower_bound, upper_bound) in color_ranges.items():
        mask = cv2.inRange(hsv_frame, np.array(lower_bound), np.array(upper_bound))
        mask = refine_mask(mask)

        if color == 'white':
            
            kernel = np.ones((5, 5), np.uint8)
            mask = cv2.erode(mask, kernel, iterations=2)
            mask = cv2.dilate(mask, kernel, iterations=2)

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, dp=1.2, minDist=20,
                                       param1=50, param2=30, minRadius=27, maxRadius=40)

            if circles is not None:
                circles = np.round(circles[0, :]).astype("int")
                for (x, y, r) in circles:
                    if mask[y, x] > 0:  
                        cx, cy = x, y
                        quadrant = get_quadrant(cx, cy, quadrants)
                        ball_id = f"{color}_ball"

                        if ball_id in ball_positions:
                            prev_quad, entry_time = ball_positions[ball_id]
                            if quadrant != prev_quad and (timestamp - entry_time) >= 4:
                                event_log.append((timestamp, prev_quad, color, 'Exit'))
                                event_log.append((timestamp, quadrant, color, 'Entry'))
                                ball_positions[ball_id] = (quadrant, timestamp)
                        else:
                            event_log.append((timestamp, quadrant, color, 'Entry'))
                            ball_positions[ball_id] = (quadrant, timestamp)

                        # Visualizing the centre of a countour 
                        cv2.circle(frame, (cx, cy), r, (0, 255, 0), 2)
                        cv2.putText(frame, f"{color}", (cx - r, cy - r), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        else:
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            for contour in contours:
                if cv2.contourArea(contour) > 1000:  # Min size of contour to filter noise
                    (x, y), radius = cv2.minEnclosingCircle(contour)
                    center = (int(x), int(y))
                    radius = int(radius)

                    quadrant = get_quadrant(center[0], center[1], quadrants)
                    ball_id = f"{color}_ball"

                    if ball_id in ball_positions:
                        prev_quad, entry_time = ball_positions[ball_id]
                        if quadrant != prev_quad and (timestamp - entry_time) >= 4:
                            event_log.append((timestamp, prev_quad, color, 'Exit'))
                            event_log.append((timestamp, quadrant, color, 'Entry'))
                            ball_positions[ball_id] = (quadrant, timestamp)
                    else:
                        event_log.append((timestamp, quadrant, color, 'Entry'))
                        ball_positions[ball_id] = (quadrant, timestamp)

                    # Drawing for visualization
                    cv2.circle(frame, center, radius, (0, 255, 0), 2)
                    cv2.putText(frame, f"{color}", (center[0] - radius, center[1] - radius), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [25]:
log_path = r'D:\System 69\New folder\events.txt'

with open(log_path, 'w') as log_file:
    log_file.write("Time, Quadrant Number, Ball Colour, Event Type\n")
    for event in event_log:
        log_file.write(f"{event[0]:.2f}, {event[1]}, {event[2]}, {event[3]}\n")


In [28]:
output_video_path = r'D:\System 69\New folder\dre.mp4'  # Update with your output video path
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

cap = cv2.VideoCapture(video_path)
while True:
    ret, frame = cap.read()
    if not ret:
        break

    

    out.write(frame)

cap.release()
out.release()
